## FML 09
***

In [ ]:
from sklearn.datasets import load_digits 
from sklearn.model_selection import train_test_split

import numpy as np

import matplotlib.pyplot as plt
import pandas as pd


%load_ext autoreload
%autoreload 2

### Kernalized Ridge Regression

### RANSAC

### Algebraic Distance

### Levenberg-Marquardt